# MLForecast Spot Price Forecasting
with LinearRegression

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [2]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('data_NO2.csv', sep=';')
df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])

df.rename(columns={'datetime_utc': 'ds'}, inplace=True)
df['unique_id'] = 1  # Assuming we only have one time series
df.drop(['volume_demand', 'volume_production'], axis=1, inplace=True)

df.head()

,ds,spot_price,unique_id
0,2015-12-31 23:00:00+00:00,16.39,1
1,2016-01-01 00:00:00+00:00,16.04,1
2,2016-01-01 01:00:00+00:00,15.74,1
3,2016-01-01 02:00:00+00:00,15.57,1
4,2016-01-01 03:00:00+00:00,15.47,1


In [4]:
train_size = int(len(df) * 0.9)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [5]:
model = MLForecast(
    models=[LinearRegression()],
    freq='h',  # hourly frequency
    lags= [1, 2, 3],  # Lags to use as features
    target_transforms=[Differences([24])],  # Transform target variable
) # TODO: maybe change the Difference-transform value. Right now, the model predicts the same every 24 hours

# Fit the model
model.fit(train, target_col='spot_price')

MLForecast(models=[LinearRegression], freq=h, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [11]:
# Make predictions
predictions = model.predict(24)
predictions.head()

,unique_id,ds,LinearRegression
0,1,2018-06-06 12:00:00+00:00,46.720942
1,1,2018-06-06 13:00:00+00:00,46.919909
2,1,2018-06-06 14:00:00+00:00,46.715788
3,1,2018-06-06 15:00:00+00:00,47.116409
4,1,2018-06-06 16:00:00+00:00,47.211833


In [7]:
test.set_index('ds', inplace=True)

In [12]:
predictions.set_index('ds', inplace=True)

In [13]:
def plot_forecasts(forecasts) -> None:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train['ds'], y=train['spot_price'], name='Train'))
    fig.add_trace(go.Scatter(x=test.index, y=test['spot_price'], name='Test'))
    fig.add_trace(go.Scatter(x=test.index, y=forecasts['LinearRegression'], name='Forecast'))
    fig.update_layout(template='simple_white', font=dict(size=18), title_text='MLForecast', 
                      width=950, title_x=0.5, height=400, xaxis_title='Date', yaxis_title='Spot Price')
    return fig.show()

plot_forecasts(predictions)

### Evaluation

In [14]:
from statsmodels.tools.eval_measures import rmse, meanabs

# calculating the root mean squared error and the mean absolute error
print(f'RMSE: ', rmse(test['spot_price'][:24], predictions['LinearRegression']))
print(f'Mean Absolute Error: ', meanabs(test['spot_price'][:24], predictions['LinearRegression']))

RMSE:  0.8921705423441372
Mean Absolute Error:  0.7414765918002454
